### Iteration Tools

__Built-In__
- `iter, reversed, next, len, slicing, zip, filter, sorted, enumerate, min, max, sum, all, any, map`

__`functools`__
- `reduce`

__`itertools`__
- Slicing:
    - `islice`
- Selecting/Filtering:
    - `dropwhile, takewhile, compress, filterfalse`
- Chaining/Teeing
    - `chain, tee`
- Map/Reduce
    - `starmap, accumulate`
- Infinite Iterators
    - `count, cycle, repeat`
- Zipping
    - `zip_longest`
- Combinatorics
    - `product, permutations, combinations, combinations_with_replacement`
    
### Aggregators

Functions that iterate through an iterable and return a single value that usually takes into account every element of the iterable.

Examples of aggregators are `min()`, `max()`, and `sum()`

__Associated Truth Values__

Every object in Python has an associated truth value of `True`, __except:__
- `None`
- `False`
- 0 in any numeric type
- empty sequences
- empty mapping types
- custom classes that implement a `__bool__` or `__len__` method that returns `False` or 0

__`any` and `all`__

`any`: returns true if any element in iterable is truthy

`all`: returns true if all elements in iterable are truthy

Often, we are not interested in the direct truthy-ness of the elements in an iterable, but reather if they satisfy some condition.

We can make `any` and `all` more useful by first applying a predicate to each element. A predicate is simply a function that takes a single argument and returns a boolean.


In [3]:
l = [1, 2, 3, 4, 100]

pred = lambda x: x < 10
less10_gen = (pred(num) for num in l)

all(less10_gen)

False

### Slicing Iterables

__`itertools.islice`__

Slicing is restricted to only sequence types, but we can use the `islice` method to slice general iterables (including iterators). __Note__ that `islice` returns a *lazy iterator*. 

In [4]:
from itertools import islice

In [7]:
l = (num for num in [1, 2, 3, 4])

res = islice(l, 0, 2)
list(res)

[1, 2]

### Selecting/Filtering

__`filter`__

Returns all elements of iterable where predicate is true. The predicate can be None, in which case the truthy-ness of the element is used to filter. __Note__ that `filter` returns a *lazy iterator*.

In [11]:
l = [1, 2, 3, 0]

res = filter(None, l)
print(list(res))

# We can achieve the same thing using a generator expresssion
res = (item for item in l if bool(item))
print(list(res))

[1, 2, 3]
[1, 2, 3]


In [13]:
res = filter(lambda x: x < 4, [0, 1, 2, 3, 4, 10])

print(list(res))

[0, 1, 2, 3]


__`itertools.filterfalse`__

Works the same way as filter function, but instead of retaining elements where the pedicate is true, it retains elements where the predicate is false. __Note__ `filterfalse` returns a *lazy iterator*.

__`itertools.compress`__

Filter an iterable, using the truthy-ness of items in another iterable. __Note__ `compress` returns a *lazy iterator*.

In [14]:
from itertools import compress

In [15]:
data = ['a', 'b', 'c', 'd']
selectors = [1, 0, 1, 1]

print(list(compress(data, selectors)))

['a', 'c', 'd']


__`itertools.takewhile`__

Returns an iterator that will yield items while the predicate is truthy. The iterator will be exhausted once encountering a falsey value, even if there are more truthy items in the iterable. __Note__ `takewhile` returns a *lazy iterator*.

In [16]:
from itertools import takewhile

In [18]:
res = takewhile(lambda x: x < 5, [0, 1, 2, 5, 2, 1])

print(list(res))

[0, 1, 2]


__`itertools.dropwhile`__

Returns an iterator that will *start* iterating (and yielding the remaining elements) once the predicate becomes falsey. The first falsey value is essentially the starting point of iteration. __Note__ `dropwhile` returns a *lazy iterator*.

In [19]:
from itertools import dropwhile

In [20]:
res = dropwhile(lambda x: x < 5, [1, 2, 3, 5, 0, 0, 0])

print(list(res))

[5, 0, 0, 0]


### Infinite Iterators

__`itertools.count`__

Works similar to `range`, although it does not have a 'stop' parameter and iterates infinitely.

__`itertools.cycle`__

Allows us to iterate over a finite iterable indefinitely.

__`itertools.repeat`__

Yields the same value indefinitely. Note that the items yielded by `repeat` is the same object in memory.

### Chaining and Teeing

__`itertools.chain`__

Analogous to sequence concatenation, although deals with iterables and lazily iterates. Has a variable number of positional arguments, each arg must be an iterable.

In [24]:
# We can manually chain iterables
l = [1, 2, 3]
iter1, iter2, iter3 = iter(l), iter(l), iter(l)

def chain_iters():
    for item in (iter1, iter2, iter3):
        yield from item

print(list(chain_iters()))

[1, 2, 3, 1, 2, 3, 1, 2, 3]


In [26]:
# Or use the chain function
iter1, iter2, iter3 = iter(l), iter(l), iter(l)

for item in chain(iter1, iter2, iter3):
    print(item)

1
2
3
1
2
3
1
2
3


In [34]:
iter1, iter2, iter3 = iter(l), iter(l), iter(l)

# If we have an iterable of iterators, theres no quick way to pass them to chain
l = [iter1, iter2, iter3]

# We can use unpacking, but unpacking is eager, and will exhaust the iterators.
chain(*l)

#Instead we can use the chain_from_iterable function
chain.from_iterable(l)

__`itertools.tee`__

Returns a tuple of copies of an iterator, for when we need to iterate through the same iterator multiple time, or in parallel.

In [35]:
from itertools import tee

In [36]:
iter_ = iter([1, 2, 3])

iters = tee(iter_, 5)
print(iters)

(<itertools._tee object at 0x7f5ad6676700>, <itertools._tee object at 0x7f5ad66760c0>, <itertools._tee object at 0x7f5ad66765c0>, <itertools._tee object at 0x7f5ad6676780>, <itertools._tee object at 0x7f5ad6676980>)


### Mapping and Reducing

Mapping: applying a callable to each element of an iterable

Accumulation: reducing an iterable down to a single value

__`map(fn, iterable)`__

Applies `fn` to every element of `iterable` and returns a lazy iterator. `fn` must be a callable that requires a single argument.

In [38]:
map(lambda x: x**2, [1, 2, 3, 4])

# We can also achieve the same thing with a generator expression
sq = lambda x: x**2
mapped = (sq(item) for item in [1, 2, 3, 4])

__`reduce(fn, iterable, [initializer])`__

Applies `fn` to each element, where `fn` is a function that takes two arguments and returns a single value. The first arguemnt will be the last value returned by the function and the second argument will be the next item in the iterable.

In [40]:
from functools import reduce

In [42]:
l = [1, 2, 3, 4]

reduce(lambda x, y: x + y, l, 100)

110

__`itertools.starmap`__

Similar to `map`, but accepts nested iterables, and will apply the function to every sub-element of the nested iterable. Useful for mapping a multi-argument function on an iterable of iterables.

In [44]:
from itertools import starmap
import operator

In [47]:
l = [ [1, 2], [3, 4] ]

#mul is a function that accepts two arguments
res = starmap(operator.mul, l)

list(res)

[2, 12]

__`itertools.accumulate(iterable, fn)`__

Similar to `reduce`, although returns a lazy iterator that produces all the intermediate results, while `reduce` only returns the final result. Unlike reduce, it does not accept an initializer. Also, the function argument is optional and will default to addition.

In [48]:
from itertools import accumulate

In [50]:
l = [1, 2, 3, 4]

reduced = reduce(operator.mul, l)
print(reduced)

accumulated = accumulate(l, operator.mul)
for result in accumulated:
    print(result)

24
1
2
6
24


###  Zipping

__`zip`__

Accepts a variable number of positional arguments, each of which are iterables. Returns an iterator that produces tuples containing elements of each iterable. Stops when one of the iterables has been completely iterated over.

In [52]:
list(zip([1, 2, 3], [10, 20], ['a', 'b', 'c', 'd']))

[(1, 10, 'a'), (2, 20, 'b')]

__`itertools.zip_longest`__

Zips based on the longest iterable, requires a default value for missing items from shorter iterables.

In [53]:
from itertools import zip_longest

In [54]:
list(zip_longest([1, 2, 3], [10, 20], ['a', 'b', 'c', 'd'], fillvalue=None))

[(1, 10, 'a'), (2, 20, 'b'), (3, None, 'c'), (None, None, 'd')]